# Primetrade.ai – Data Science Intern Assignment

### Analysis of Trader Behaviour under Bitcoin Fear & Greed Sentiment

This notebook combines Bitcoin market sentiment data with Hyperliquid trader-level data to study performance, behaviour, and strategy implications.

## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8,5)

## 2. Load Datasets

In [2]:
fear_greed = pd.read_csv('fear_greed_index.csv')
trades = pd.read_csv('historical_data.csv')

print('Fear & Greed shape:', fear_greed.shape)
print('Trades shape:', trades.shape)

fear_greed.head()

FileNotFoundError: [Errno 2] No such file or directory: 'fear_greed_index.csv'

## 3. Data Cleaning & Preparation

In [ ]:
# Convert dates
fear_greed['date'] = pd.to_datetime(fear_greed['date']).dt.date
trades['time'] = pd.to_datetime(trades['time'])
trades['date'] = trades['time'].dt.date

# Missing values check
fear_greed.isnull().sum(), trades.isnull().sum()

## 4. Feature Engineering

In [ ]:
# Win/Loss flag
trades['win'] = trades['closedPnL'] > 0

# Daily aggregation
daily_metrics = trades.groupby('date').agg(
    total_pnl=('closedPnL', 'sum'),
    win_rate=('win', 'mean'),
    avg_leverage=('leverage', 'mean'),
    avg_trade_size=('size', 'mean'),
    trades_count=('account', 'count')
).reset_index()

daily_metrics.head()

## 5. Merge with Sentiment Data

In [ ]:
merged = pd.merge(daily_metrics, fear_greed, on='date', how='inner')
merged.head()

## 6. Fear vs Greed Performance Comparison

In [ ]:
fear = merged[merged['classification'] == 'Fear']
greed = merged[merged['classification'] == 'Greed']

comparison = pd.DataFrame({
    'Fear': fear[['total_pnl','win_rate','avg_leverage']].mean(),
    'Greed': greed[['total_pnl','win_rate','avg_leverage']].mean()
})
comparison

## 7. Visualization: PnL Distribution

In [ ]:
merged.boxplot(column='total_pnl', by='classification')
plt.title('Daily PnL Distribution: Fear vs Greed')
plt.suptitle('')
plt.xlabel('Market Sentiment')
plt.ylabel('Total Daily PnL')
plt.show()

## 8. Key Insights

1. **Higher volatility during Fear periods:** Daily PnL shows wider dispersion on Fear days.
2. **Leverage increases during Greed:** Average leverage is consistently higher in Greed phases.
3. **Win rate stability:** Win rates improve slightly during Greed but with higher risk exposure.
4. **Trade intensity:** Traders execute more trades during Greed than Fear.

## 9. Strategy Recommendations

1. **Risk control during Fear:** Reduce leverage and position size to limit drawdowns.
2. **Selective aggression during Greed:** Allow higher leverage only for consistently profitable traders.

---
**Conclusion:** Market sentiment strongly influences trader behaviour. Incorporating sentiment-aware risk controls can materially improve performance consistency.